In [1]:
import sys
import torch
import click
import json
import datetime
from timeit import default_timer as timer

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils import data
import torchvision
import types

from tqdm import tqdm
from tensorboardX import SummaryWriter

# from models.gradient_scaler import MinNormElement
import losses
import datasets
import metrics
import model_selector
from min_norm_solvers import MinNormSolver, gradient_normalizers

NUM_EPOCHS = 100

In [2]:
with open('../configs.json') as config_params:
    configs = json.load(config_params)

with open('../mnist.json') as json_params:
    params = json.load(json_params)


exp_identifier = []
for (key, val) in params.items():
    if 'tasks' in key:
        continue
    if 'scales' in key:
        continue
    exp_identifier+= ['{}={}'.format(key,val)]
    
exp_identifier = '|'.join(exp_identifier)
params['exp_id'] = exp_identifier

In [3]:
from models.multi_lenet import MultiLeNetO, MultiLeNetR

model = {}
model['rep'] = MultiLeNetR()
model['rep'].cuda()
model['L'] = MultiLeNetO()
model['L'].cuda()
model['R'] = MultiLeNetO()
model['R'].cuda()

MultiLeNetO(
  (fc1): Linear(in_features=50, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [4]:
train_loader, train_dst, val_loader, val_dst = datasets.get_dataset(params, configs)

In [5]:
# from loaders.multi_mnist_loader import MNIST
# from torchvision import transforms
# def global_transformer():
#     return transforms.Compose([transforms.ToTensor(),
#                                transforms.Normalize((0.1307,), (0.3081,))])

# dst = MNIST(root='../datasets/mnist/', train=True, download=True, transform=global_transformer(), multi=True)
# loader = torch.utils.data.DataLoader(dst, batch_size=10, shuffle=True)

In [12]:
from livelossplot import PlotLosses
liveloss = PlotLosses(fig_path='test.pdf')
model_params = []
for m in model:
    model_params += model[m].parameters()
optimizer = torch.optim.RMSprop(model_params, lr=params['lr'])

for batch in train_loader:
#     for m in model:
#     print(batch)
    images = batch[0]
    images = Variable(images.cuda())
#         model[m].train()
    optimizer.zero_grad()
    labels = batch[1]
#     print(labels)
    labels = Variable(labels.cuda())
    images_volatile = Variable(images.data, volatile=True)
    mask = None
    rep, mask = model['rep'].forward(images_volatile, mask)
    loss = losses.nll(rep, mask)
    loss.backward()
    optimizer.step()
    logs["loss"] = loss.detach()
    liveloss.update(logs)
    liveloss.draw()

/users/tr.zaiqm/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target'

In [14]:
mask

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         ...,

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 

In [ ]:
len(batch)

In [ ]:
batch[0].shape

In [ ]:
batch[1].shape

In [ ]:
batch[2].shape